<a href="https://colab.research.google.com/github/nessb26/certificate_tutorial_week6_aglo/blob/main/TestProject_week7_AlgoTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from pylab import plt
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score

In [2]:
raw = pd.read_csv('http://hilpisch.com/pyalgo_eikon_eod_data.csv',
                          index_col=0, parse_dates=True).dropna()
raw.head(3)


,AAPL.O,MSFT.O,INTC.O,AMZN.O,GS.N,SPY,.SPX,.VIX,EUR=,XAU=,GDX,GLD
Date,,,,,,,,,,,,
2010-01-04,30.572827,30.95,20.88,133.90,173.08,113.33,1132.99,20.04,1.4411,1120.00,47.71,109.80
2010-01-05,30.625684,30.96,20.87,134.69,176.14,113.63,1136.52,19.35,1.4368,1118.65,48.17,109.70
2010-01-06,30.138541,30.77,20.80,132.25,174.26,113.71,1137.14,19.16,1.4412,1138.50,49.34,111.51


In [56]:
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 23 19:11:21 2022

@author: nessi
"""


import numpy as np
import pandas as pd
from pylab import plt
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score

class BackTest_Event:
  def __init__(self, symbol, start, end, amount,ftc=0.0, ptc=0.0, verbose=True):
        self.symbol = symbol
        self.start = start
        self.end = end
        self.initial_amount = amount  
        self.amount = amount  
        self.ftc = ftc  
        self.ptc = ptc  
        self.units = 0  
        self.position = 0  
        self.trades = 0  
        self.verbose = verbose  
        self.get_data()
        self.get_features(splitRatio=0.7,lags=5)
        self.accuracy={}
        self.accuracy["accuracy_test"]={}
        self.accuracy["accuracy_train"]={}
  
  def reinit(self,amount):
    self.initial_amount = amount  
    self.amount = amount  
    self.units = 0  
    self.position = 0  
    self.trades = 0 


  def get_date_price(self, bar):
        ''' Return date and price for bar. '''
        date = str(self.data.index[bar])[:10]
        price = self.data.price.iloc[bar]
        return date, price

  def print_balance(self, bar):
        ''' Print out current cash balance info.'''
        date, price = self.get_date_price(bar)
        print(f'{date} | current balance {self.amount:.2f}')

  def print_net_wealth(self, bar):
        ''' Print out current cash balance info.    '''
        date, price = self.get_date_price(bar)
        net_wealth = self.units * price + self.amount
        print(f'{date} | current net wealth {net_wealth:.2f}')

  def place_buy_order(self, bar, units=None, amount=None):
        ''' Place a buy order.
        '''
        date, price = self.get_date_price(bar)  
        if units is None:  
            units = int(amount / price)  
        self.amount -= (units * price) * (1 + self.ptc) + self.ftc  
        self.units += units  
        self.trades += 1  
        if self.verbose:  
            print(f'{date} | buying {units} units at {price:.2f}')  
            self.print_balance(bar)  
            self.print_net_wealth(bar)  
  
  def place_sell_order(self, bar, units=None, amount=None):
        ''' Place a sell order.
        '''
        date, price = self.get_date_price(bar)
        if units is None:
            units = int(amount / price)
        self.amount += (units * price) * (1 - self.ptc) - self.ftc  
        self.units -= units  
        self.trades += 1
        if self.verbose:
            print(f'{date} | selling {units} units at {price:.2f}')
            self.print_balance(bar)
            self.print_net_wealth(bar)
  
  def close_out(self, bar):
        ''' Closing out a long or short position.
        '''
        date, price = self.get_date_price(bar)
        self.amount += self.units * price
        self.units = 0
        self.trades += 1
        if self.verbose:
            print(f'{date} | inventory {self.units} units at {price:.2f}')
            print('=' * 55)
        print('Final balance   [$] {:.2f}'.format(self.amount))
        perf = ((self.amount - self.initial_amount) /
                self.initial_amount * 100)
        print('Net Performance [%] {:.2f}'.format(perf))
        print('Trades Executed [#] {:.2f}'.format(self.trades))
  
  def go_long(self, bar, units=None, amount=None):  
        if self.position == -1:  
            self.place_buy_order(bar, units=-self.units)  
        if units:  
            self.place_buy_order(bar, units=units)  
        elif amount:  
            if amount == 'all':  
                amount = self.amount  
            self.place_buy_order(bar, amount=amount)  

  def go_short(self, bar, units=None, amount=None):
        if self.position == 1:
            self.place_sell_order(bar, units=self.units)
        if units:
            self.place_sell_order(bar, units=units)
        elif amount:
            if amount == 'all':
                amount = self.amount
            self.place_sell_order(bar, amount=amount)

  def get_data(self):
        ''' Retrieves and prepares the data.
        '''
        url='http://hilpisch.com/pyalgo_eikon_eod_data.csv'
        #url = 'http://hilpisch.com/ref_eikon_eod_data.csv'
        raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()
        raw = pd.DataFrame(raw[self.symbol])
        raw = raw.loc[self.start:self.end]
        raw.rename(columns={self.symbol: 'price'}, inplace=True)
        self.data = raw.dropna()
  
  def get_indicators(self,SMA1=20,SMA2=60):
    self.data['r'] = np.log(self.data["price"] / self.data["price"].shift(1))
    self.data['d'] = np.where(self.data['r'] > 0, 1, -1)

    bins = [-0.0075, -0.005, 0.005, 0.0075]
    self.data['d_'] = np.digitize(self.data['r'], bins=bins)
    
    self.data['SMA1'] = self.data["price"].rolling(SMA1).mean()
    self.data['SMA2'] = self.data["price"].rolling(SMA2).mean()
    self.data['SMA_'] = self.data['SMA1'] - self.data['SMA2']
    self.data['EWMA1'] = self.data["price"].ewm(halflife=SMA1).mean()
    self.data['EWMA2'] = self.data["price"].ewm(halflife=SMA2).mean()
    self.data['EWMA_'] = self.data['EWMA1'] - self.data['EWMA2']
    self.data['V1'] = self.data['r'].rolling(SMA1).std()
    self.data['V2'] = self.data['r'].rolling(SMA2).std()
    self.data.dropna(inplace=True)

  def normalize(self,x, mu, std):
    return (x - mu) / std

  def get_features(self, SMA1=20,SMA2=60,splitRatio=0.7,lags=5):
    self.get_indicators(SMA1=20,SMA2=60)
    
    split = int(len(self.data) * splitRatio)
    self.train = self.data.iloc[:split].copy()
    self.test = self.data.iloc[split:].copy()
    
    cols = ['r','d','d_','SMA1','SMA2','SMA_','EWMA1','EWMA2','EWMA_','V1','V2']
    #exclude for normalisation
    exclude = ['r', 'd', 'd_', 'V1', 'V2']
    #exclude = [""]

    #normalisation
    mu, std = self.train.mean(), self.train.std()

    cols_ = list()
    for col in cols:
      for lag in range(1, lags + 1):
        col_ = col + f'_lag_{lag}'
        if col not in exclude:
           # Lag feature and normalize using mu and std previously calculated
          self.train[col_] = self.normalize(self.train[col].shift(lag),mu[col], std[col])
          self.test[col_] = self.normalize(self.test[col].shift(lag),mu[col], std[col])
        else:
          # Lag feature
          self.train[col_] = self.train[col].shift(lag)
          self.test[col_] = self.test[col].shift(lag)
        cols_.append(col_)

    self.train.dropna(inplace=True)
    self.test.dropna(inplace=True)
    self.features=cols_
    
  def backtest(self,m):
    models = {'gauss': GaussianNB(),
          'logreg': LogisticRegression(C=1, solver='lbfgs', max_iter=500),
          'dtc': DecisionTreeClassifier(max_depth=7),
          'svm': SVC(C=1, gamma='auto', kernel='linear'),
          'mlp': MLPClassifier(hidden_layer_sizes=[32], shuffle=False,
                              max_iter=1000)}
    model = models[m]
    model.fit(self.train[self.features], self.train['d'])
    
    self.train['p_' + m] = model.predict(self.train[self.features])
    self.train['p_' + m] = np.where(self.train['p_' + m] > 0, 1, -1)
    self.accuracy["accuracy_train"][m]=accuracy_score(self.train['d'],self.train['p_' + m])

    self.test['p_' + m] = model.predict(self.test[self.features])
    self.test['p_' + m] = np.where(self.test['p_' + m] > 0, 1, -1)
    self.accuracy["accuracy_test"][m]=accuracy_score(self.test['d'],self.test['p_' + m])

    for bar in range(0, len(self.test)):
      if self.position == 0:  
        if (self.test['p_' + m].iloc[bar]==1):  
          self.go_long(bar, amount=self.initial_amount)  
          self.position = 1  
        elif (self.test['p_' + m].iloc[bar] == -1):  
          self.go_short(bar, amount=self.initial_amount)
          self.position = -1  
      elif self.position == 1:  
          if (self.test['p_' + m].iloc[bar]==-1):  
            self.place_sell_order(bar, units=2*self.units)  
            self.position = 1  
      elif self.position == -1:  
        if (self.test['p_' + m].iloc[bar]==1):  
            self.place_buy_order(bar, units=-2*self.units)  
            self.position = 1
      
    self.close_out(bar)



In [70]:
symbol="EUR="
start="2010-01-01"
end="2019-12-31"

amount=100.0
ftc=0.00
ptc=0.01
verbose=False
models=['gauss','logreg','dtc','svm','mlp']
a=BackTest_Event(symbol, start, end, amount,ftc, ptc, verbose)

for m in models:
  print("Backtest model "+m)
  a.backtest(m)
  print('Accuracy train   {:.3f}'.format(a.accuracy["accuracy_train"][m]))
  print('Accuracy test    {:.3f}'.format(a.accuracy["accuracy_test"][m]))
  print('=' * 55)
  a.reinit(amount=amount)




Backtest model gauss
Final balance   [$] 96.63
Net Performance [%] -3.37
Trades Executed [#] 565.00
Accuracy train   0.536
Accuracy test    0.507
Backtest model logreg
Final balance   [$] 96.82
Net Performance [%] -3.18
Trades Executed [#] 210.00
Accuracy train   0.543
Accuracy test    0.496
Backtest model dtc
Final balance   [$] 89.42
Net Performance [%] -10.58
Trades Executed [#] 338.00
Accuracy train   0.603
Accuracy test    0.536
Backtest model svm
Final balance   [$] 88.07
Net Performance [%] -11.93
Trades Executed [#] 210.00
Accuracy train   0.538
Accuracy test    0.507
Backtest model mlp
Final balance   [$] 105.54
Net Performance [%] 5.54
Trades Executed [#] 429.00
Accuracy train   0.621
Accuracy test    0.492


In [55]:
a.data.d_.value_counts()

2    1711
0     194
1     193
4     180
3     178
Name: d_, dtype: int64

In [30]:
a.train

,price,r,d,SMA1,SMA2,SMA_,EWMA1,EWMA2,EWMA_,V1,...,V2_lag_1,V2_lag_2,V2_lag_3,V2_lag_4,V2_lag_5,p_gauss,p_logreg,p_dtc,p_svm,p_mlp
Date,,,,,,,,,,,,,,,,,,,,,
2010-04-08,1.3358,0.001348,1,1.351830,1.373253,-0.021423,1.362577,1.372985,-0.010407,0.006456,...,0.005964,0.006035,0.006038,0.006007,0.005958,1,1,0,1,0
2010-04-09,1.3496,0.010278,1,1.350925,1.371590,-0.020665,1.362087,1.372486,-0.010399,0.006928,...,0.005876,0.005964,0.006035,0.006038,0.006007,1,1,0,1,0
2010-04-12,1.3584,0.006499,1,1.350040,1.370047,-0.020007,1.361949,1.372189,-0.010241,0.006948,...,0.006065,0.005876,0.005964,0.006035,0.006038,1,1,0,1,0
2010-04-13,1.3606,0.001618,1,1.349705,1.368553,-0.018848,1.361898,1.371947,-0.010049,0.006828,...,0.006140,0.006065,0.005876,0.005964,0.006035,1,-1,0,-1,0
2010-04-14,1.3651,0.003302,1,1.349115,1.367335,-0.018220,1.362018,1.371805,-0.009787,0.006670,...,0.006149,0.006140,0.006065,0.005876,0.005964,1,-1,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-01-20,1.0699,0.003370,1,1.054300,1.067270,-0.012970,1.067284,1.090260,-0.022976,0.005478,...,0.005621,0.005624,0.005564,0.005487,0.005470,-1,-1,1,-1,1
2017-01-23,1.0761,0.005778,1,1.055990,1.067062,-0.011072,1.067584,1.090097,-0.022513,0.005543,...,0.005641,0.005621,0.005624,0.005564,0.005487,-1,1,1,1,1
2017-01-24,1.0730,-0.002885,0,1.057465,1.066767,-0.009302,1.067768,1.089901,-0.022132,0.005632,...,0.005694,0.005641,0.005621,0.005624,0.005564,-1,1,0,1,1


In [47]:
len(a.data)

2456

In [16]:
a.test.head()

,price,r,d,SMA1,SMA2,SMA_,EWMA1,EWMA2,EWMA_,V1,...,V2_lag_1,V2_lag_2,V2_lag_3,V2_lag_4,V2_lag_5,p_gauss,p_logreg,p_dtc,p_svm,p_mlp
Date,,,,,,,,,,,,,,,,,,,,,
2016-09-27,1.1213,-0.003561,0,1.120595,1.115857,0.004738,1.119529,1.118058,0.001471,0.003632,...,0.003852,0.003846,0.003899,0.003921,0.004060,-1,-1,-1,-1,-1
2016-09-28,1.1214,0.000089,1,1.120960,1.116088,0.004872,1.119593,1.118096,0.001496,0.003487,...,0.003868,0.003852,0.003846,0.003899,0.003921,-1,1,-1,1,-1
2016-09-29,1.1221,0.000624,1,1.121285,1.116292,0.004993,1.119678,1.118142,0.001536,0.003479,...,0.003799,0.003868,0.003852,0.003846,0.003899,-1,-1,-1,-1,-1
2016-09-30,1.1238,0.001514,1,1.121495,1.116590,0.004905,1.119818,1.118207,0.001611,0.003406,...,0.003791,0.003799,0.003868,0.003852,0.003846,-1,1,-1,1,1
2016-10-03,1.1209,-0.002584,0,1.121775,1.116852,0.004923,1.119855,1.118238,0.001617,0.003339,...,0.003762,0.003791,0.003799,0.003868,0.003852,-1,1,-1,1,-1


In [66]:
models = {'gauss': GaussianNB(),
          'logreg': LogisticRegression(C=1, solver='lbfgs', max_iter=500),
          'dtc': DecisionTreeClassifier(max_depth=7),
          'svm': SVC(C=1, gamma='auto', kernel='linear'),
          'mlp': MLPClassifier(hidden_layer_sizes=[32], shuffle=False,
                              max_iter=1000)}

modelss=['gauss','logreg','dtc','svm','mlp']
for m in modelss:
  model = models[m]
  model.fit(a.train[a.features], a.train['d'])
    
  a.train['p_' + m] = model.predict(a.train[a.features])
  print("model "+m)
  print(a.train['p_' + m].head(3))
  print('=' * 55)
    #self.train['p_' + m] = np.where(self.train['p_' + m] > 0, 1, -1)
    #self.accuracy["accuracy_train"][m]=accuracy_score(self.train['d'],self.train['p_' + m])

model gauss
Date
2010-04-08    1
2010-04-09    1
2010-04-12    1
Name: p_gauss, dtype: int64
model logreg
Date
2010-04-08    1
2010-04-09    1
2010-04-12    1
Name: p_logreg, dtype: int64
model dtc
Date
2010-04-08   -1
2010-04-09   -1
2010-04-12   -1
Name: p_dtc, dtype: int64
model svm
Date
2010-04-08    1
2010-04-09    1
2010-04-12    1
Name: p_svm, dtype: int64
model mlp
Date
2010-04-08   -1
2010-04-09    1
2010-04-12   -1
Name: p_mlp, dtype: int64
